In [8]:
import os
import time
import concurrent.futures
import cv2
import math
import numpy as np
from PIL import Image
import imutils


def enlarge_image(image, scale_factor):
    height, width = image.shape[:2]
    return cv2.resize(image, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)


def resize(filename, output_filename):
    try:
        img = cv2.imread(filename)
        if img is None:
            raise ValueError(f"[ERROR] Cannot read image: {filename}")

        width, height = img.shape[1], img.shape[0]

        # 작은 이미지일 경우 확대
        if height * width * 3 <= 2 ** 20:
            scale_factor = 2.0
            enlarged = enlarge_image(img, scale_factor)
            cv2.imwrite(output_filename, enlarged)
            return cv2.imread(output_filename)

        # 큰 이미지일 경우 축소
        elif height * width * 3 > 2 ** 25:
            i = 2
            t_height, t_width = height, width
            while t_height * t_width * 3 > 2 ** 25:
                t_height = int(t_height / math.sqrt(i))
                t_width = int(t_width / math.sqrt(i))
                i += 1
            resized = cv2.resize(img, (t_width, t_height))
            cv2.imwrite(output_filename, resized)
            return cv2.imread(output_filename)

        # 적절한 크기면 원본 반환
        else:
            return img

    except Exception as e:
        print(f"[EXCEPTION in resize()] {filename} => {e}")
        return None


def preprocess_image(image):
    if image is None:
        return None
    return cv2.GaussianBlur(image, (5, 5), 0)


def split_image(image, num_rows, num_cols):
    height, width = image.shape[:2]
    split_images = []

    cell_height = height // num_rows
    cell_width = width // num_cols

    for r in range(num_rows):
        for c in range(num_cols):
            start_x = c * cell_width
            end_x = start_x + cell_width
            start_y = r * cell_height
            end_y = start_y + cell_height
            split = image[start_y:end_y, start_x:end_x]
            split_images.append(split)

    return split_images


def stitch_images(images):
    stitcher = cv2.createStitcher() if imutils.is_cv3() else cv2.Stitcher_create()
    (status, stitched) = stitcher.stitch(images)

    if status == 0:
        output_path = "C:\\Users\\yjyoo\\result_image.jpg"
        cv2.imwrite(output_path, stitched)
        print(f"[INFO] Stitched image saved at: {output_path}")
        return stitched
    else:
        print(f"[ERROR] Stitching failed with status code: {status}")
        return None


def assign_work(files, num_workers):
    work_per_worker = len(files) // num_workers
    work_assignments = []
    for i in range(num_workers):
        start_index = i * work_per_worker
        end_index = (i + 1) * work_per_worker if i < num_workers - 1 else len(files)
        worker_files = files[start_index:end_index]
        work_assignments.append(worker_files)
    return work_assignments


def main():
    DIR = "C:\\Users\\yjyoo\\pic"
    files = [f for f in os.listdir(DIR) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

    if not files:
        print("[ERROR] No image files found in directory.")
        return

    start = time.time()

    num_workers = 4
    work_assignments = assign_work(files, num_workers)

    resized_images = []

    print("[INFO] Resizing images with multithreading...")

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = []
        for batch in work_assignments:
            for filename in batch:
                full_path = os.path.join(DIR, filename)
                output_path = full_path.replace(".", "_resized.")
                futures.append(executor.submit(resize, full_path, output_path))

        for future in concurrent.futures.as_completed(futures):
            try:
                result = future.result()
                if result is not None:
                    resized_images.append(result)
            except Exception as e:
                print(f"[ERROR] Exception during resizing: {e}")

    print(f"[INFO] {len(resized_images)} images resized successfully.")

    # 이미지 전처리
    preprocessed_images = []
    for img in resized_images:
        processed = preprocess_image(img)
        if processed is not None:
            preprocessed_images.append(processed)

    print(f"[INFO] {len(preprocessed_images)} images preprocessed.")

    # 이미지 분할
    num_rows = 2
    num_cols = 2
    split_images_list = []
    for image in preprocessed_images:
        split_images_list.extend(split_image(image, num_rows, num_cols))

    print(f"[INFO] Total {len(split_images_list)} image chunks created.")

    # 스티칭
    print("[INFO] Stitching images...")
    stitched_image = stitch_images(preprocessed_images[:5])  # 또는 전체 preprocessed_images 사용

    end = time.time()

    if stitched_image is not None:
        print(f"[INFO] Stitching complete. Time taken: {end - start:.2f} seconds.")
    else:
        print("[INFO] Stitching failed.")


if __name__ == '__main__':
    main()


[INFO] Resizing images with multithreading...
[INFO] 23 images resized successfully.
[INFO] 23 images preprocessed.
[INFO] Total 92 image chunks created.
[INFO] Stitching images...
[INFO] Stitched image saved at: C:\Users\yjyoo\result_image.jpg
[INFO] Stitching complete. Time taken: 33.75 seconds.
